# estimation.feols { #pyfixest.estimation.feols }

`estimation.feols(fml, data, vcov=None, ssc=ssc(), fixef_rm='none', collin_tol=1e-10, drop_intercept=False, i_ref1=None, i_ref2=None)`

Estimate linear regression models with fixed effects using fixest formula syntax.

This method accommodates complex models with stepwise regressions, multiple dependent variables, interaction of variables,
interacted fixed effects, and instruments. It's compatible with various syntax elements from the formulaic module.

## Parameters

| Name             | Type                                         | Description                                                                                                                                                                                                                                                                                                                                                          | Default    |
|------------------|----------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------|
| `fml`            | str                                          | A three-sided formula string using fixest formula syntax. Syntax: "Y ~ X1 + X2 \| FE1 + FE2 \| X1 ~ Z1". "\|" separates dependent variable, fixed effects, and instruments. Special syntax includes stepwise regressions, cumulative stepwise regression, multiple dependent variables, interaction of variables (i(X1,X2)), and interacted fixed effects (fe1^fe2). | _required_ |
| `data`           | pyfixest.dev_utils.DataFrameType             | A pandas or polars dataframe containing the variables in the formula.                                                                                                                                                                                                                                                                                                | _required_ |
| `vcov`           | typing.Union\[str, dict\[str, str\]\]        | Type of variance-covariance matrix for inference. Options include "iid", "hetero", "HC1", "HC2", "HC3", or a dictionary for CRV1/CRV3 inference.                                                                                                                                                                                                                     | `None`     |
| `ssc`            | str                                          | A ssc object specifying the small sample correction for inference.                                                                                                                                                                                                                                                                                                   | `ssc()`    |
| `fixef_rm`       | str                                          | Specifies whether to drop singleton fixed effects. Options: "none" (default), "singleton".                                                                                                                                                                                                                                                                           | `'none'`   |
| `collin_tol`     | float                                        | Tolerance for collinearity check, by default 1e-06.                                                                                                                                                                                                                                                                                                                  | `1e-10`    |
| `drop_intercept` | bool                                         | Whether to drop the intercept from the model, by default False.                                                                                                                                                                                                                                                                                                      | `False`    |
| `i_ref1`         | typing.Optional\[typing.Union\[list, str\]\] | Reference category for the first set of categorical variables interacted via "i()", by default None.                                                                                                                                                                                                                                                                 | `None`     |
| `i_ref2`         | typing.Optional\[typing.Union\[list, str\]\] | Reference category for the second set of categorical variables interacted via "i()", by default None.                                                                                                                                                                                                                                                                | `None`     |

## Returns

| Type   | Description                                                                                      |
|--------|--------------------------------------------------------------------------------------------------|
| object | An instance of the `Feols` class or `FixestMulti` class for multiple models specified via `fml`. |

## Examples

As in `fixest`, the `feols` function can be used to estimate a simple linear regression model with fixed effects.
The following example regresses `Y` on `X1` and `X2` with fixed effects for `f1` and `f2`: fixed effects are specified
after the `|` symbol.


In [ ]:
from pyfixest.estimation import feols
from pyfixest.utils import get_data
from pyfixest.summarize import etable

data = get_data()

fit = feols("Y ~ X1 + X2 | f1 + f2", data)
fit.summary()

Calling `feols()` returns an instance of the `Feols` class. The `summary()` method can be used to print the results.

An alternative way to retrieve model results is via the `tidy()` method, which returns a pandas dataframe with the
estimated coefficients, standard errors, t-statistics, and p-values.


In [ ]:
fit.tidy()

You can also access all elements in the tidy data frame by dedicated methods, e.g. `fit.coef()` for the coefficients, `fit.se()` for the standard errors,
`fit.tstat()` for the t-statistics, and `fit.pval()` for the p-values, and `fit.confint()` for the confidence intervals.

The employed type of inference can be specified via the `vcov` argument. If vcov is not provided, `PyFixest` employs the `fixest` default of iid inference,
unless there are fixed effects in the model, in which case `feols()` clusters the standard error by the first fixed effect (CRV1 inference).


In [ ]:
fit1 = feols("Y ~ X1 + X2 | f1 + f2", data, vcov="iid")
fit2 = feols("Y ~ X1 + X2 | f1 + f2", data, vcov="hetero")
fit3 = feols("Y ~ X1 + X2 | f1 + f2", data, vcov={"CRV1": "f1"})

Supported inference types are "iid", "hetero", "HC1", "HC2", "HC3", and "CRV1"/"CRV3". Clustered standard errors are specified via a dictionary, e.g. `{"CRV1": "f1"}`
for CRV1 inference with clustering by `f1` or `{"CRV3": "f1"}` for CRV3 inference with clustering by `f1`. For two-way clustering, you can provide a formula string, e.g.
`{"CRV1": "f1 + f2"}` for CRV1 inference with clustering by `f1`.


In [ ]:
fit4 = feols("Y ~ X1 + X2 | f1 + f2", data, vcov={"CRV1": "f1 + f2"})

Inference can be adjusted post estimation via the `vcov` method:


In [ ]:
fit.summary()
fit.vcov("iid").summary()

The `ssc` argument specifies the small sample correction for inference. In general, `feols()` uses all of `fixest::feols()` defaults, but sets the
`fixef.K` argument to `"none"` whereas the `fixest::feols()` default is `"nested"`. See here for more details: [link to github](https://github.com/s3alfisc/pyfixest/issues/260).

`feols()` supports a range of multiple estimation syntax, i.e. you can estimate multiple models in one call. The following example estimates two models, one with
fixed effects for `f1` and one with fixed effects for `f2` using the `sw()` syntax.


In [ ]:
fit = feols("Y ~ X1 + X2 | sw(f1, f2)", data)
type(fit)

The returned object is an instance of the `FixestMulti` class. You can access the results of the first model via `fit.fetch_model(0)` and the results of the second model
via `fit.fetch_model(1)`. You can compare the model results via the `etable()` function:


In [ ]:
etable([fit.fetch_model(0), fit.fetch_model(1)])

Other supported multiple estimation syntax include `sw0()`, `csw()` and `csw0()`. While `sw()` adds variables in a "stepwise" fashinon, `csw()`
does so cumulatively.


In [ ]:
fit = feols("Y ~ X1 + X2 | csw(f1, f2)", data)
etable([fit.fetch_model(0), fit.fetch_model(1)])

The `sw0()` and `csw0()` syntax are similar to `sw()` and `csw()`, but start with a model that exludes the variables specified in `sw()` and `csw()`:


In [ ]:
fit = feols("Y ~ X1 + X2 | sw0(f1, f2)", data)
etable([fit.fetch_model(0), fit.fetch_model(1), fit.fetch_model(2)])

The `feols()` function also supports multiple dependent variables. The following example estimates two models, one with `Y1` as the dependent variable and one with `Y2` as the dependent variable.


In [ ]:
fit = feols("Y + Y2 ~ X1 | f1 + f2", data)
etable([fit.fetch_model(0), fit.fetch_model(1)])

It is possible to combine different multiple estimation operators:


In [ ]:
fit = feols("Y + Y2 ~ X1 | sw(f1, f2)", data)
etable([fit.fetch_model(0), fit.fetch_model(1), fit.fetch_model(2), fit.fetch_model(3)])

In general, using muliple estimation syntax can improve the estimation time as covariates that are demeaned in one model and are used
in another model do not need to be demeaned again: `feols()` implements a caching mechanism that stores the demeaned covariates.

Besides OLS, `feols()` also supports IV estimation via three part formulas:


In [ ]:
fit = feols("Y ~  X2 | f1 + f2 | X1 ~ Z1", data)
fit.tidy()

Here, `X1` is the endogenous variable and `Z1` is the instrument. `f1` and `f2` are the fixed effects, as before. To estimate
IV models without fixed effects, simply omit the fixed effects part of the formula:


In [ ]:
fit = feols("Y ~  X2 | X1 ~ Z1", data)
fit.tidy()

Last, `feols()` supports interaction of variables via the `i()` syntax. Documentation on this is tba.

After fitting a model via `feols()`, you can use the `predict()` method to get the predicted values:


In [ ]:
fit = feols("Y ~ X1 + X2 | f1 + f2", data)
fit.predict()[0:5]

The `predict()` method also supports a `newdata` argument to predict on new data, which returns a numpy array of the predicted values:


In [ ]:
fit = feols("Y ~ X1 + X2 | f1 + f2", data)
fit.predict(newdata=data)[0:5]

Last, you can plot the results of a model via the `coefplot()` method:


In [ ]:
fit = feols("Y ~ X1 + X2 | f1 + f2", data)
fit.coefplot()